<h1> Основная работа и тестирование датасета ADE20K. </h1>
<h6> Взяты с <a href='https://github.com/CSAILVision/ADE20K/'>этого источника</a><h6>

In [ ]:
%load_ext autoreload
%autoreload 2
import IPython.display
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pickle as pkl
from ADE20K_utils import *


In [ ]:
# Загрузка индекса с глобальной информацией о датасете
DATASET_PATH = '/home/jupyter/mnt/datasets/ADE20K'
index_file = '/ADE20K_2021_17_01/index_ade20k.pkl'
with open('{}/{}'.format(DATASET_PATH, index_file), 'rb') as f:
    index_ade20k = pkl.load(f)


In [ ]:
print("Файл был загружен. Вывожу описание атрибутов: ")
print('--------------------------------------------')
for attribute_name, desc in index_ade20k['description'].items():
    print('* {}: {}'.format(attribute_name, desc))
print('--------------------------------------------\n')

i = 1686 # 16899, 16964
nfiles = len(index_ade20k['filename'])
file_name = index_ade20k['filename'][i]
num_obj = index_ade20k['objectPresence'][:, i].sum()
num_parts = index_ade20k['objectIsPart'][:, i].sum()
count_obj = index_ade20k['objectPresence'][:, i].max()
obj_id = np.where(index_ade20k['objectPresence'][:, i] == count_obj)[0][0]
obj_name = index_ade20k['objectnames'][obj_id]
full_file_name = '{}/{}'.format(index_ade20k['folder'][i], index_ade20k['filename'][i])
print("В датасете {} картинок".format(nfiles))
print("Картинка с индексом {} это {}".format(i, file_name))
print("Она расположена в {}".format(full_file_name))
print("Происходит на сцене: {}".format(index_ade20k['scene'][i]))
print("Имеет {} объектов, из которых {} - это части".format(num_obj, num_parts))
print("Самый частый объект это {} ({}), он появляется в количестве {}".format(obj_name, obj_id, count_obj))



In [ ]:
root_path = DATASET_PATH

# Эта функция считывает файлы изображений и масок и создает экземпляры и маски сегментации.
info = loadAde20K('{}/{}'.format(root_path, full_file_name))
img = cv2.imread(info['img_name'])[:,:,::-1]
seg = cv2.imread(info['segm_name'])[:,:,::-1]
seg_mask = seg.copy()

print(info['img_name'])

# 0 индекс обозначает задний фон (нераспределённые объекты)
seg_mask[info['class_mask'] != obj_id+1] *= 0
plt.figure(figsize=(15,5))

plt.imshow(np.concatenate([img, seg, seg_mask], 1))
plt.axis('off')
if len(info['partclass_mask']):
    plt.figure(figsize=(5*len(info['partclass_mask']), 5))
    plt.title('Части')
    plt.imshow(np.concatenate(info['partclass_mask'],1))
    plt.axis('off')
    

In [ ]:
instances = [4, 10, 19]
images = []
for instance in instances:
    file_instance = '{}/{}/instance_{:03}_{}'.format(
        root_path, full_file_name.replace('.jpg', ''), instance, file_name.replace('.jpg', '.png'))
    aux = cv2.imread(file_instance)
    images.append(aux)
plt.figure(figsize=(5*len(images), 5))
plt.imshow(np.concatenate(images, 1))
plt.axis('off')


In [ ]:
img = plot_polygon('{}/{}'.format(root_path, full_file_name), info)
plt.figure(figsize=(15,15))
plt.imshow(img[:, :, ::-1])
plt.axis('off')


In [ ]:
info = loadAde20K('{}/{}'.format(root_path, full_file_name))

# Загрузка маски
mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)

# Получение значения класса для конкретного пикселя (x, y)
x = 100  # Координата x пикселя
y = 200  # Координата y пикселя

class_id = mask[y, x]

print(f"Значение класса для пикселя ({x}, {y}): {class_id}")